# 實價買賣資料
## 處理事項
- 由實價登錄網站抓取預售買賣資料，每月更新，更新時間同預售屋社區資料
- 資料清洗

In [ ]:
import os
import re
import time
import datetime
from typing import Generator, List, Any, Literal
import pandas as pd
import requests
from tqdm import tqdm
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.configs import SALE_BASE_URL, SALE_URLS_FRAGMENTS
from utils.helper_function import build_complete_urls, combined_df, csv_extractor

## 從實價登錄網站取得實價買賣資料
- function
    - build_complete_url(網址組合)、fetch_data(向實價網址請求資料)、combined_df(合併全台預售屋資料)

In [ ]:
# 組合「實價買賣」網址
urls = build_complete_urls(SALE_BASE_URL, SALE_URLS_FRAGMENTS)
for idx, (city, link) in enumerate(urls.items(), start=1):
    print(f"{idx}. {city} → {link}")

print(f"\n總共有 {len(urls)} 筆縣市資料")

In [ ]:
# 依上述網址向實價登錄網站請求資料
raw_bs_transaction_df = combined_df(urls, "20250801")

In [ ]:
# 儲存原始資料
output_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\bs_transaction\raw"
os.makedirs(output_dir, exist_ok=True)

csv_fn = "raw_bs_transaction_20250801.csv"
out_path = os.path.join(output_dir,  csv_fn)
raw_bs_transaction_df.to_csv(out_path, index=False, encoding='utf-8-sig')

## 載入原始資料並進行後續資料清洗
- function：csv_extractor(csv載入)

In [ ]:
input_dir = r"C:\pylabs\housing-market-insights\data\lvr_moi\bs_transaction\raw"
csv_fn = "raw_bs_transaction_20250801.csv"
input_path = os.path.join(input_dir,  csv_fn)

In [ ]:
# 載入原始預售屋社區資料
extracted =csv_extractor(input_path)
print(f" 逐筆交易資料載入成功: {extracted.shape}  記憶體使用: {extracted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
extracted